In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
import utils

In [7]:
import sys
from pathlib import Path
from itertools import chain, product

import dotenv
import numpy as np
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.orm import aliased

from api import db, make_app
from api import models, utils
from api.blp import BlpQuery
from api.manager import InstrumentManager, Clone
from api.inputs import Instruments, InstrumentUI

app = make_app()
app.app_context().push()

In [3]:
root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

### Bloomberg Connection

In [ ]:
bq = BlpQuery().start()

In [6]:
tickers = pd.read_csv(tickers_csv, header=None).squeeze().rename(None)

In [41]:
df = pd.read_sql(models.Bond.query.statement, db.engine)

In [42]:
tickers[~tickers.isin(df['ticker'])]

11       BW433738 Corp
30       BW299707 Corp
32       BW462003 Corp
33       BW299705 Corp
35       BW358643 Corp
             ...      
12469    EJ798438 Corp
12470    BW646036 Corp
12471    BW733144 Corp
12472    BW742597 Corp
12473    BW741347 Corp
Length: 1077, dtype: object

In [28]:
r1 = aliased(models.Rating)
r2 = aliased(models.Rating)
r3 = aliased(models.Rating)
r4 = aliased(models.Rating)

In [37]:
q = db.session.query(
    models.Bond.ticker,
    models.Bond.security_name,
    models.Country.iso.label('iso_country'),
    models.Country.name,
    models.Currency.iso.label('iso_currency'),
    models.Bond.isin,
    models.Bond.date_issue,
    models.Bond.date_maturity,
    models.Bond.date_calc_maturity,
    models.Bond.maturity_type,
    models.Bond.issue_amount,
    models.Bond.coupon,
    models.Bond.coupon_type,
    models.Bond.coupon_frequency,
    models.Bond.day_count,
    models.Bond.is_fixed,
    models.Bond.is_callable,
    models.Bond.is_convertible,
    models.Bond.is_perpetual,
    models.Bond.is_subordinated,
    models.Bond.normlized_payment_rank,
    models.Bond.bail_in_bond_designation,
    r1.composite.label('rating_sp'),
    r3.composite.label('rating_fitch'),
    r2.composite.label('rating_moodys'),
    r4.composite.label('rating_bbg'),
    
).join(
   r1, sa.and_(
        r1.id == models.Bond.id_rating_sp,
        sa.or_(r1.provider == 'sp', r1.provider == None)
   )
).join(
   r3, sa.and_(
        r3.id == models.Bond.id_rating_fitch,
        sa.or_(r3.provider == 'fitch', r3.provider == None)
   )
).join(
   r2, sa.and_(
        r2.id == models.Bond.id_rating_moodys,
        sa.or_(r2.provider == 'moodys', r2.provider == None)
   )
).join(
   r4, sa.and_(
        r4.id == models.Bond.id_rating_bbg,
        sa.or_(r4.provider == 'bbg', r2.provider == None)
   )
).join(
    models.Country,
    models.Country.id == models.Bond.id_country_of_risk
).join(
    models.Currency,
    models.Currency.id == models.Bond.id_currency
).join(
    models.Issuer,
    models.Issuer.id == models.Bond.id_issuer
)

In [38]:
reference_data = pd.read_sql(q.statement, db.engine)

ticker          security_name iso_country  \
0      BV996920 Corp         BAC 6 1/8 PERP          US   
1      BV979451 Corp      MS 5.297 04/20/37          US   
2      BV990770 Corp     JPM 4.586 04/26/33          US   
3      BV979449 Corp       MS 4.21 04/20/28          US   
4      BV979641 Corp     WFC 4.611 04/25/53          US   
...              ...                    ...         ...   
14856  AV387586 Corp     AGN 2 5/8 11/15/28          US   
14857  AV609067 Corp     ABIBB 4.9 02/01/46          BE   
14858  AL012673 Corp      VZ 3 1/8 11/02/35          US   
14859  EJ892414 Corp  RESFER 3 1/8 10/25/28          FR   
14860  BR949913 Corp     SAH 4 5/8 11/15/29          US   

                                 name iso_currency          isin date_issue  \
0      United States Of America (The)          USD  US06055HAB96 2022-04-22   
1      United States Of America (The)          USD  US61747YES00 2022-04-20   
2      United States Of America (The)          USD  US46647PDC77 2022-04-26   
3      United States Of America (The)          USD  US61747YER27 2022-04-20   
4      United States Of America (The)          USD  US95000U2Z51 2022-04-25   
...                               ...          ...           ...        ...   
14856  United States Of America (The)          EUR  XS1909193317 2018-11-15   
14857                         Belgium          USD  US03522AAF75 2018-11-13   
14858  United States Of America (The)          GBP  XS1405769727 2016-11-02   
14859                          France          EUR  XS0985666436 2013-10-25   
14860  United States Of America (The)          USD  USU83546AJ74 2021-10-27   

      date_maturity date_calc_maturity maturity_type  ...  is_callable  \
0               NaT         2170-07-27     PERP/CALL  ...         True   
1        2037-04-20         2037-04-20      CALLABLE  ...         True   
2        2033-04-26         2033-04-26      CALLABLE  ...         True   
3        2028-04-20         2028-04-20      CALLABLE  ...         True   
4        2053-04-25         2053-04-25      CALLABLE  ...         True   
...             ...                ...           ...  ...          ...   
14856    2028-11-15         2028-11-15      CALLABLE  ...         True   
14857    2046-02-01         2046-02-01      CALLABLE  ...         True   
14858    2035-11-02         2035-11-02   AT MATURITY  ...        False   
14859    2028-10-25         2028-10-25   AT MATURITY  ...        False   
14860    2029-11-15         2029-11-15      CALLABLE  ...         True   

       is_convertible is_perpetual is_subordinated     normlized_payment_rank  \
0                True        False            True  Jr Subordinated Unsecured   
1               False        False            True     Subordinated Unsecured   
2               False        False           False               Sr Unsecured   
3               False        False           False               Sr Unsecured   
4               False        False           False               Sr Unsecured   
...               ...          ...             ...                        ...   
14856           False        False           False               Sr Unsecured   
14857           False        False           False               Sr Unsecured   
14858           False        False           False               Sr Unsecured   
14859           False        False           False               Sr Unsecured   
14860           False        False           False               Sr Unsecured   

       bail_in_bond_designation  rating_sp  rating_fitch  rating_moodys  \
0                Not Applicable       BBB-          BBB+           None   
1      Insufficient Information        BBB          BBB+           None   
2                             Y         A-          None           None   
3                             Y       BBB+             A           None   
4      Insufficient Information       BBB+            A+           None   
...                         ...        ...  

In [ ]:
db.session.query().join

In [ ]:
db.session.query().join

In [ ]:
b.query.get(1).rating_sp

In [ ]:
bs = models.Bond.query.all()

In [ ]:
for bo in bs:
    for attr in ['id_rating_sp', 'id_rating_fitch', 'id_rating_moodys', 'id_rating_bbg', 'id_rating_dbrs']:
        attr_ = getattr(bo, attr)
        if not attr_:
            setattr(bo, attr, 147)

In [ ]:
bo.id_rating_moodys

In [ ]:
db.session.add_all(bs)

In [ ]:
db.session.commit()